## Deliverable 2. Create a Customer Travel Destinations Map.

In [75]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [76]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dhanaula,IN,30.2833,75.5833,65.80,50,0,4.09,clear sky
1,1,Berezovyy,RU,51.6667,135.7000,-6.68,89,19,4.54,few clouds
2,2,Busselton,AU,-33.6500,115.3333,62.10,74,0,12.50,clear sky
3,3,Jamestown,US,42.0970,-79.2353,42.39,75,97,17.00,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,58.48,93,58,3.00,light rain


In [77]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [106]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
28,28,Hilo,US,19.7297,-155.0900,74.28,90,90,5.75,light rain
29,29,Barranca,PE,-10.7500,-77.7667,70.84,73,91,9.86,overcast clouds
37,37,Sur,OM,22.5667,59.5289,74.66,59,0,3.06,clear sky
38,38,Puerto Ayora,EC,-0.7393,-90.3518,73.36,82,89,8.34,overcast clouds
50,50,Langsa,ID,4.4683,97.9683,74.35,93,64,2.46,broken clouds
61,61,Carnarvon,AU,-24.8667,113.6333,73.47,83,0,8.05,clear sky
67,67,Rio Grande,BR,-32.0350,-52.0986,74.95,73,100,16.17,overcast clouds
72,72,Naberera,TZ,-4.2000,36.9333,73.20,54,90,14.61,overcast clouds
77,77,Lucapa,AO,-8.4192,20.7447,70.65,87,90,2.68,light rain
101,101,Avarua,CK,-21.2078,-159.7750,73.45,100,75,2.30,light rain


In [107]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                160
City                   160
Country                159
Lat                    160
Lng                    160
Max Temp               160
Humidity               160
Cloudiness             160
Wind Speed             160
Current Description    160
dtype: int64

In [108]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.count()

City_ID                159
City                   159
Country                159
Lat                    159
Lng                    159
Max Temp               159
Humidity               159
Cloudiness             159
Wind Speed             159
Current Description    159
dtype: int64

In [109]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
28,Hilo,US,74.28,light rain,19.7297,-155.0900,
29,Barranca,PE,70.84,overcast clouds,-10.7500,-77.7667,
37,Sur,OM,74.66,clear sky,22.5667,59.5289,
38,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,
50,Langsa,ID,74.35,broken clouds,4.4683,97.9683,
...,...,...,...,...,...,...,...
1947,Ilo,PE,71.02,overcast clouds,-17.6394,-71.3375,
1990,Likasi,CD,74.32,scattered clouds,-10.9814,26.7333,
1992,Valdosta,US,73.69,scattered clouds,30.8327,-83.2785,
1993,Catalina Foothills,US,74.16,few clouds,32.2979,-110.9187,


In [110]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [134]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df.loc[(hotel_df["Hotel Name"] == ""), "Hotel Name"] = np.nan
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
28,Hilo,US,74.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
29,Barranca,PE,70.84,overcast clouds,-10.7500,-77.7667,El Olivar Hospedaje
37,Sur,OM,74.66,clear sky,22.5667,59.5289,Sur Plaza Hotel
38,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
50,Langsa,ID,74.35,broken clouds,4.4683,97.9683,OYO 2964 Hotel Ridho
61,Carnarvon,AU,73.47,clear sky,-24.8667,113.6333,Hospitality Carnarvon
67,Rio Grande,BR,74.95,overcast clouds,-32.0350,-52.0986,Hotel Atlântico Rio Grande
77,Lucapa,AO,70.65,light rain,-8.4192,20.7447,Residência Ulisses Dos Santos
101,Avarua,CK,73.45,light rain,-21.2078,-159.7750,Paradise Inn
102,Alta Floresta,BR,73.51,light rain,-9.8756,-56.0861,Floresta Amazonica Hotel


In [135]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [136]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>County</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [139]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))